In [4]:
%load_ext autoreload
%autoreload 2

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import shutil
import time
from keras.models import Model, Sequential
from tqdm.auto import tqdm
from IPython.display import clear_output
from pathlib import Path

from PIL import Image

from metrics import gmsd, psnr, ssim
from models.spectral_encoder import recon_model, ssim_loss
from loader.dataloader import DataLoader
from reg.reg_d import RegD

DATA_DIR = "/Users/farhanoktavian/imperial/thesis/sandbox/dataset/raw"
METADATA_DIR = "/Users/farhanoktavian/imperial/thesis/sandbox/dataset/data.h5"
STACKREG_DIR = Path("/Users/farhanoktavian/imperial/thesis/sandbox/dataset/stackreg3")

run_loader = DataLoader(DATA_DIR, METADATA_DIR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
sample = run_loader.samples[0]
run = sample.runs[2]
print(f"{sample.sample_id} - {run.run_id}")

HS001 - Run06


In [6]:
target_wavelength = 600

reg = RegD()

start_t = time.time()

for c in range(run.collection_count):
    print(f"Collection {c}/{run.collection_count}")
    imgs, img_paths = run.get_spectral_images(collection_idx=c, with_path=True)
    tgt_img = run.get_spectral_image(wavelength=target_wavelength, collection_idx=c)
    for i in range(len(imgs)):
        print(f"Image {i+1}/{len(imgs)}")
        img = imgs[i]
        img_path = img_paths[i]
        img = reg.register(img, tgt_img)

        save_path = STACKREG_DIR / sample.sample_id / run.run_id
        save_path.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(save_path / img_path.name), img)

end_t = time.time()
print(f"Time taken: {end_t - start_t}s")

Collection 0/1
Image 1/29
Image 2/29
Image 3/29
Image 4/29
Image 5/29
